In [2]:
if True:
    import os
    os.environ["PYSPARK_SUBMIT_ARGS"]='--packages com.databricks:spark-csv_2.10:1.3.0 pyspark-shell'
    execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))
    import os
    import sys
else:
    os.environ['SPARK_HOME'] = '/usr/lib/spark'
    sys.path.insert(0, '/usr/lib/spark/python/lib/py4j-0.9-src.zip')
    sys.path.insert(0, '/usr/lib/spark/python/')
    sys.path.insert(0, '/usr/local/lib64/python2.7/site-packages')
    sys.path.insert(0,'/usr/local/lib/python2.7/site-packages')
    
    from pyspark import SparkContext
    from pyspark.sql import SQLContext, HiveContext
    
    try: sc = SparkContext()
    except: None    
    sqlc = SQLContext(sc)
    spark = sqlc

In [4]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [5]:
import seaborn as sns

In [6]:
import numpy as np
import pandas as pd
import time
import json

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, MapType
import pyspark.sql.functions as F

In [7]:
from operator import add
from pyspark.sql.types import *

In [8]:
from pyspark.mllib.recommendation import Rating
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel
from datetime import datetime

# Load data

In [10]:
strain_predictions = spark.read.parquet("lab_12/cache/strain_bp_v1")

In [11]:
strain_predictions.show(2)

+------+-------+------+----------------+--------------+--------------+-----------+-----------+------------------+--------------------+
|userId|movieId|rating|user_sum_ratings|n_user_ratings|n_item_ratings|nr_user_cat|nr_item_cat|         bp_rating|       bp_rating_err|
+------+-------+------+----------------+--------------+--------------+-----------+-----------+------------------+--------------------+
|180844|     31|   4.0|           133.0|            33|           109|  unr_06_50|    inr_101| 4.335332651046607|-0.33533265104660703|
|170647|     31|   4.5|           696.5|           179|           109|     unr_51|    inr_101|4.2530035613100505| 0.24699643868994947|
+------+-------+------+----------------+--------------+--------------+-----------+-----------+------------------+--------------------+
only showing top 2 rows



In [12]:
stest_predictions = spark.read.parquet("lab_12/cache/stest_bp_v1")

In [13]:
stest_predictions.show(2)

+------+-------+------+----------------+--------------+--------------+-----------+-----------+------------------+-------------------+
|userId|movieId|rating|user_sum_ratings|n_user_ratings|n_item_ratings|nr_user_cat|nr_item_cat|         bp_rating|      bp_rating_err|
+------+-------+------+----------------+--------------+--------------+-----------+-----------+------------------+-------------------+
| 52854|     31|   4.0|           431.0|         125.0|         109.0|     unr_51|    inr_101|3.8228106894522753| 0.1771893105477247|
| 88479|     31|   3.0|           831.5|         269.0|         109.0|     unr_51|    inr_101|3.4709176327304334|-0.4709176327304334|
+------+-------+------+----------------+--------------+--------------+-----------+-----------+------------------+-------------------+
only showing top 2 rows



# Calculate normalised user rating per movie

In [15]:
avg_rating = strain_predictions.agg({"rating":"mean"}).collect()[0][0]
print "avg_rating", avg_rating

avg_rating 3.5214315006


In [16]:
strain_predictions = strain_predictions.withColumn("user_avg_rating", 
                        (F.col("user_sum_ratings") + F.lit(avg_rating)) / (F.col("n_user_ratings") + F.lit(1.0)))
strain_predictions = strain_predictions.withColumn("user_norm_rating", F.col("rating") - F.col("user_avg_rating"))
strain_predictions = strain_predictions.select("userId", "movieId", "user_norm_rating").cache()

In [17]:
strain_predictions.show(2)

+------+-------+--------------------+
|userId|movieId|    user_norm_rating|
+------+-------+--------------------+
|180844|     31|-0.01533622060588...|
|170647|     31|  0.6109920472188888|
+------+-------+--------------------+
only showing top 2 rows



In [18]:
strain_predictions.registerTempTable("strain")

In [19]:
stest_predictions = stest_predictions.withColumn("user_avg_rating", 
                        (F.col("user_sum_ratings") + F.lit(avg_rating)) / (F.col("n_user_ratings") + F.lit(1.0)))
stest_predictions = stest_predictions.select("userId", "movieId", "rating", 
                                             "user_avg_rating", "nr_user_cat", "nr_item_cat", "bp_rating").cache()

In [20]:
stest_predictions.show(2)

+------+-------+------+------------------+-----------+-----------+------------------+
|userId|movieId|rating|   user_avg_rating|nr_user_cat|nr_item_cat|         bp_rating|
+------+-------+------+------------------+-----------+-----------+------------------+
| 52854|     31|   4.0|3.4485827896873014|     unr_51|    inr_101|3.8228106894522753|
| 88479|     31|   3.0|3.0926719685207407|     unr_51|    inr_101|3.4709176327304334|
+------+-------+------+------------------+-----------+-----------+------------------+
only showing top 2 rows



In [21]:
stest_predictions.registerTempTable("stest")

In [22]:
#stest_predictions.cache()

DataFrame[userId: int, movieId: int, rating: float, user_avg_rating: double, nr_user_cat: string, nr_item_cat: string, bp_rating: double]

# Load similarity matrix

In [26]:
def convert_u2u_tupple(line):
    r = line.split(',')
    return (int(r[0]), int(r[1]), float(r[2]), int(r[3]))

def load_u2u_sim():
    raw = sc.textFile("lab_12/u2u_top100to200.csv")\
            .filter(lambda x: not x.startswith("user"))\
            .map(convert_u2u_tupple)
    return raw.toDF(schema=StructType([StructField("user1",  IntegerType()),
                                       StructField("user2", IntegerType()),
                                       StructField("sim",  FloatType()),
                                       StructField("sim_rank",  IntegerType())])) 
    return raw

In [27]:
u2u = load_u2u_sim()

In [28]:
u2u.show(2)

+-----+------+----------+--------+
|user1| user2|       sim|sim_rank|
+-----+------+----------+--------+
|    1|179650|0.93893325|       0|
|    1|227097|0.93850267|       1|
+-----+------+----------+--------+
only showing top 2 rows



In [29]:
u2u.cache()

DataFrame[user1: int, user2: int, sim: float, sim_rank: int]

In [30]:
u2u.registerTempTable("u2u_sim")

In [32]:
strain_predictions.dtypes

[('userId', 'int'), ('movieId', 'int'), ('user_norm_rating', 'double')]

# Make predictions on test

In [74]:
q = spark.sql("""
        select
          pr.userId,
          pr.movieId,
          avg(
              if(pr.user_avg_rating < 0.001, cast({avg_rating} as double), pr.user_avg_rating)
          ) + 
              sum(kr.user_norm_rating * s.sim) /  sum(s.sim)  as predicted_rating,
          count(*) as n_used_ratings
        from stest pr
        join u2u_sim s on pr.userId = s.user1 and s.sim_rank <= 150
        join strain kr on pr.movieId = kr.movieId and kr.userId = s.user2
        group by pr.userId, pr.movieId
""".format(avg_rating=avg_rating))

In [75]:
q.show(2)

+------+-------+------------------+--------------+
|userId|movieId|  predicted_rating|n_used_ratings|
+------+-------+------------------+--------------+
|  5993|      6|3.8953496049892538|             4|
| 76127|     48| 4.676986890932069|            14|
+------+-------+------------------+--------------+
only showing top 2 rows



In [76]:
predicted_on_test = q

In [77]:
predicted_on_test.registerTempTable("predicted_test")

In [78]:
eval_data = spark.sql("""
        select
          tbase.*,
          coalesce(pt.n_used_ratings, 0) as n_used_user_ratings,
          coalesce(predicted_rating, bp_rating) as predicted_rating,
          if(pt.n_used_ratings is not null, 1, 0) as has_u2u_rating
        from stest tbase
        left join predicted_test  pt on tbase.userId = pt.userId and tbase.movieId = pt.movieId
""")

In [79]:
eval_data.cache()

DataFrame[userId: int, movieId: int, rating: float, user_avg_rating: double, nr_user_cat: string, nr_item_cat: string, bp_rating: double, n_used_user_ratings: bigint, predicted_rating: double, has_u2u_rating: int]

In [80]:
eval_data.show(2)

+------+-------+------+------------------+-----------+-----------+-----------------+-------------------+------------------+--------------+
|userId|movieId|rating|   user_avg_rating|nr_user_cat|nr_item_cat|        bp_rating|n_used_user_ratings|  predicted_rating|has_u2u_rating|
+------+-------+------+------------------+-----------+-----------+-----------------+-------------------+------------------+--------------+
|    28|   4711|   3.0| 3.279679312247347|     unr_51|    inr_101|3.564750454655524|                 14| 3.634586279581508|             1|
|    40|  13667|   3.0|3.5617033192353844|     unr_51|    inr_101|3.778930846940237|                  1|3.9114890042293844|             1|
+------+-------+------+------------------+-----------+-----------+-----------------+-------------------+------------------+--------------+
only showing top 2 rows



In [81]:
msedf=eval_data.select(F.pow(F.col("rating")-F.col("predicted_rating"), 2).alias("e2"))\
               .agg(F.avg(F.col("e2")).alias("mse"))

In [82]:
print 'rmse on all test set:', np.sqrt(msedf.collect()[0][0])

rmse on all test set: 0.813953247318


In [83]:
msedf_u2uonly=eval_data.filter(F.col("has_u2u_rating")==1) \
                       .select(F.pow(F.col("rating")-F.col("predicted_rating"), 2).alias("e2"))\
                       .agg(F.avg(F.col("e2")).alias("mse"))

In [84]:
print 'rmse on records with u2u predictions:', np.sqrt(msedf_u2uonly.collect()[0][0])

rmse on records with u2u predictions: 0.787357439438


In [85]:
msedf_u2uonly_t3=eval_data.filter(F.col("n_used_user_ratings")>=3) \
                       .select(F.pow(F.col("rating")-F.col("predicted_rating"), 2).alias("e2"))\
                       .agg(F.avg(F.col("e2")).alias("mse"))

In [86]:
print 'rmse on records with at least 3 similar u2u predictions:', np.sqrt(msedf_u2uonly_t3.collect()[0][0])

rmse on records with at least 3 similar u2u predictions: 0.743408071797


In [87]:
qg = eval_data.withColumn("e2", F.pow(F.col("rating")-F.col("predicted_rating"), 2))\
             .groupBy(["nr_user_cat", "nr_item_cat"])\
             .agg(F.avg(F.col("e2")).alias("mse"), 
                  F.count(F.col("e2")).alias("n_examples"),
                  F.sum(F.col("has_u2u_rating")).alias("has_u2u_rating"),
                  F.avg(F.col("n_used_user_ratings")).alias("avg_used_user_ratings"))

In [88]:
epg = qg.toPandas()

In [89]:
epg['r_examples'] = np.round(epg['n_examples'] * 100. /  epg['n_examples'].sum(), 2)
epg['rmse'] = np.sqrt(epg['mse'])
epg['r_has_u2u_rating'] = np.round(epg['has_u2u_rating'] * 100. /  epg['n_examples'], 2)

In [90]:
epg.sort_values(by='r_examples', ascending=False)

,nr_user_cat,nr_item_cat,mse,n_examples,has_u2u_rating,avg_used_user_ratings,r_examples,rmse,r_has_u2u_rating
0,unr_51,inr_101,0.664078,2016442,1898843,11.801940,63.81,0.814910,94.17
8,unr_06_50,inr_101,0.632496,900019,845370,12.701757,28.48,0.795296,93.93
3,unr_01_05,inr_101,0.640584,122519,108384,11.776867,3.88,0.800365,88.46
4,unr_51,inr_11_100,0.819001,88496,34498,0.650798,2.80,0.904987,38.98
11,unr_51,inr_01_10,0.825465,12069,1084,0.100754,0.38,0.908551,8.98
9,unr_06_50,inr_11_100,1.123363,11725,3021,0.426780,0.37,1.059888,25.77
13,unr_0,inr_101,1.327059,3962,3701,28.381878,0.13,1.151980,93.41
1,unr_51,inr_0,0.881028,1610,0,0.000000,0.05,0.938631,0.00
6,unr_01_05,inr_11_100,1.331763,1572,316,0.303435,0.05,1.154020,20.10
5,unr_06_50,inr_01_10,1.404502,1130,38,0.035398,0.04,1.185117,3.36


In [93]:
print eval_data.filter(F.col("n_used_user_ratings")>=3).count()
print eval_data.count()

2473596
3159915


In [94]:
eval_data.select("userId", "movieId", "rating", "nr_user_cat", "nr_item_cat", 
                 "n_used_user_ratings", "has_u2u_rating", 
                 F.col("predicted_rating").alias("predicted_u2u_rating")).show(2)

+------+-------+------+-----------+-----------+-------------------+--------------+--------------------+
|userId|movieId|rating|nr_user_cat|nr_item_cat|n_used_user_ratings|has_u2u_rating|predicted_u2u_rating|
+------+-------+------+-----------+-----------+-------------------+--------------+--------------------+
|    28|   4711|   3.0|     unr_51|    inr_101|                 14|             1|   3.634586279581508|
|    40|  13667|   3.0|     unr_51|    inr_101|                  1|             1|  3.9114890042293844|
+------+-------+------+-----------+-----------+-------------------+--------------+--------------------+
only showing top 2 rows



# Save for ensemble cheks

In [95]:
eval_data = eval_data.select("userId", "movieId", "rating", "nr_user_cat", "nr_item_cat", 
                 "n_used_user_ratings", "has_u2u_rating", 
                 F.col("predicted_rating").alias("predicted_u2u_rating"))

In [96]:
eval_data = eval_data.coalesce(8)

In [97]:
eval_data.show(2)

+------+-------+------+-----------+-----------+-------------------+--------------+--------------------+
|userId|movieId|rating|nr_user_cat|nr_item_cat|n_used_user_ratings|has_u2u_rating|predicted_u2u_rating|
+------+-------+------+-----------+-----------+-------------------+--------------+--------------------+
|    28|   4711|   3.0|     unr_51|    inr_101|                 14|             1|   3.634586279581508|
|    40|  13667|   3.0|     unr_51|    inr_101|                  1|             1|  3.9114890042293844|
+------+-------+------+-----------+-----------+-------------------+--------------+--------------------+
only showing top 2 rows



In [98]:
eval_data.write.parquet("lab_12/cache/stest_bp_u2u_top150", mode='overwrite')